### Add Microsoft.ML Package

In [0]:
#r "nuget:Microsoft.ML , 1.3.1"

In [1]:
using System;
using System.IO;
using Microsoft.ML;
using Microsoft.ML.Data;

In [2]:
public class SentimentIssue
{
  [LoadColumn(0)]
  public string Text { get; set; }
  [LoadColumn(1)]
  public bool Label { get; set; }
}

In [3]:
public class SentimentPrediction
{
   [ColumnName("PredictedLabel")]
   public bool Prediction { get; set; }
   
   public float Probability { get; set; }

   public float Score { get; set; }
}

### Create MLContext to be shared across the model creation workflow objects 

In [4]:
var mlContext = new MLContext(seed: 1);

### STEP 1: Common data loading configuration

In [5]:
IDataView trainingData = mlContext.Data.LoadFromTextFile<SentimentIssue>(@"dataset/mldataset/train_data.tsv", hasHeader: true);
IDataView testData = mlContext.Data.LoadFromTextFile<SentimentIssue>(@"dataset/mldataset/train_data.tsv", hasHeader: true);

### STEP 2: Common data process configuration with pipeline data transformations

In [6]:
var dataProcessPipeline = mlContext.Transforms.Text.FeaturizeText("Features", nameof(SentimentIssue.Text));

### STEP 3: Set the training algorithm, then create and config the modelBuilder

In [7]:
var trainer = mlContext.BinaryClassification.Trainers.LbfgsLogisticRegression("Label", "Features");
var trainingPipeline = dataProcessPipeline.Append(trainer);

### STEP 4: Train the model fitting to the DataSet

In [8]:
ITransformer trainedModel = trainingPipeline.Fit(trainingData);

### STEP 5: Evaluate the model and show accuracy stats

In [9]:
var predictions = trainedModel.Transform(testData);
var metrics = mlContext.BinaryClassification.Evaluate(data: predictions, labelColumnName: "Label", scoreColumnName: "Score");

In [10]:
Console.WriteLine($"*       Accuracy: {metrics.Accuracy:P2}");
Console.WriteLine($"*       Area Under Curve:      {metrics.AreaUnderRocCurve:P2}");
Console.WriteLine($"*       Area under Precision recall Curve:  {metrics.AreaUnderPrecisionRecallCurve:P2}");
Console.WriteLine($"*       F1Score:  {metrics.F1Score:P2}");
Console.WriteLine($"*       LogLoss:  {metrics.LogLoss:#.##}");
Console.WriteLine($"*       LogLossReduction:  {metrics.LogLossReduction:#.##}");
Console.WriteLine($"*       PositivePrecision:  {metrics.PositivePrecision:#.##}");
Console.WriteLine($"*       PositiveRecall:  {metrics.PositiveRecall:#.##}");
Console.WriteLine($"*       NegativePrecision:  {metrics.NegativePrecision:#.##}");
Console.WriteLine($"*       NegativeRecall:  {metrics.NegativeRecall:P2}");

*       Accuracy: 71.20 %

*       Area Under Curve:      79.77 %

*       Area under Precision recall Curve:  78.96 %

*       F1Score:  71.20 %

*       LogLoss:  .87

*       LogLossReduction:  .13

*       PositivePrecision:  .71

*       PositiveRecall:  .71

*       NegativePrecision:  .71

*       NegativeRecall:  71.20 %

### STEP 6: Save/persist the trained model to a .ZIP file

In [11]:
mlContext.Model.Save(trainedModel, trainingData.Schema, @"dataset/mldataset/SentimentModel.zip");


### TRY IT: Make a single test prediction loding the model from .ZIP file

In [12]:
SentimentIssue sampleStatement = new SentimentIssue { Text = "This is a very good film" };

In [13]:
var predEngine = mlContext.Model.CreatePredictionEngine<SentimentIssue, SentimentPrediction>(trainedModel);

In [14]:
var resultprediction = predEngine.Predict(sampleStatement);

In [15]:
Console.WriteLine($"=============== Single Prediction  ===============");
Console.WriteLine($"Text: {sampleStatement.Text} | Prediction: {(Convert.ToBoolean(resultprediction.Prediction) ? "Toxic" : "Non Toxic")} sentiment | Probability of being toxic: {resultprediction.Probability} ");

=============== Single Prediction  ===============

Text: This is a very good film | Prediction: Toxic sentiment | Probability of being toxic: 0.6830624 